In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


### About dataset

This is Kaggle dataset taken from Home Credit - Credit Risk Model Stability competition(https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability).

Overview
The goal of this competition is to predict which clients are more likely to default on their loans. The evaluation will favor solutions that are stable over time.

Your participation may offer consumer finance providers a more reliable and longer-lasting way to assess a potential client’s default risk.

In [2]:
# Открываем файл для чтения с использованием контекстного менеджера
with open("data_path.txt", "r") as file:
    # Читаем содержимое файла
    dataPath = file.read()
dataPath

'D:/data/kaggle_comp1/'

In [3]:
file_name = "feature_definitions.csv"

file_path = os.path.join(dataPath, file_name)

# Прочитайте CSV файл в DataFrame с помощью pandas
df = pd.read_csv(file_path)

df

,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actu...
1,actualdpdtolerance_344P,DPD of client with tolerance.
2,addres_district_368M,District of the person's address.
3,addres_role_871L,Role of person's address.
4,addres_zip_823M,Zip code of the address.
...,...,...
460,totinstallast1m_4525188A,Total amount of monthly instalments paid in th...
461,twobodfilling_608L,Type of application process.
462,type_25L,Contact type of a person.
463,typesuite_864L,Persons accompanying the client during the loa...


In [4]:
file_name = "csv_files/train/train_applprev_1_0.csv"

file_path = os.path.join(dataPath, file_name)

# Прочитайте CSV файл в DataFrame с помощью pandas
data1 = pd.read_csv(file_path, dtype={27: str})

data1

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
1,2,0.0,1682.4,NaN,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,...,1,NaN,12.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,12.0
2,3,0.0,6140.0,NaN,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,...,0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0
3,4,0.0,2556.6,NaN,NaN,P24_27_36,NaN,2019-01-08,NaN,0.0,...,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,24.0
4,5,0.0,NaN,NaN,NaN,P85_114_140,NaN,2019-01-16,NaN,NaN,...,0,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3887679,2651092,0.0,0.0,NaN,NaN,P94_109_143,NaN,2018-05-04,NaN,100000.0,...,3,NaN,NaN,P46_145_78,a55475b1,P99_56_166,P94_109_143,NaN,D,NaN
3887680,2651092,0.0,4366.0,2017-11-09,NaN,a55475b1,NaN,2017-11-09,NaN,0.0,...,4,0.0,5.0,P60_146_156,a55475b1,a55475b1,a55475b1,NaN,K,5.0
3887681,2651092,0.0,2632.2,2015-12-11,1.0,a55475b1,1.0,2015-12-11,NaN,0.0,...,5,0.0,14.0,P46_145_78,a55475b1,a55475b1,a55475b1,NaN,K,14.0
3887682,2651092,0.0,NaN,2014-02-15,60000.0,a55475b1,1.0,2014-02-15,NaN,53300.0,...,6,0.0,NaN,P46_145_78,a55475b1,a55475b1,a55475b1,680531000.0,K,NaN


In [5]:
data1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3887684 entries, 0 to 3887683
Data columns (total 41 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   case_id                      int64  
 1   actualdpd_943P               float64
 2   annuity_853A                 float64
 3   approvaldate_319D            object 
 4   byoccupationinc_3656910L     float64
 5   cancelreason_3545846M        object 
 6   childnum_21L                 float64
 7   creationdate_885D            object 
 8   credacc_actualbalance_314A   float64
 9   credacc_credlmt_575A         float64
 10  credacc_maxhisbal_375A       float64
 11  credacc_minhisbal_90A        float64
 12  credacc_status_367L          object 
 13  credacc_transactions_402L    float64
 14  credamount_590A              float64
 15  credtype_587L                object 
 16  currdebt_94A                 float64
 17  dateactivated_425D           object 
 18  district_544M                object 
 19  

In [6]:
data1.isna().sum()

case_id                              0
actualdpd_943P                    2234
annuity_853A                    155851
approvaldate_319D              1766021
byoccupationinc_3656910L       2896024
cancelreason_3545846M                0
childnum_21L                   1953893
creationdate_885D                   35
credacc_actualbalance_314A     3719506
credacc_credlmt_575A            119070
credacc_maxhisbal_375A         3719506
credacc_minhisbal_90A          3719506
credacc_status_367L            3719506
credacc_transactions_402L      3719506
credamount_590A                 123329
credtype_587L                   123329
currdebt_94A                   1270377
dateactivated_425D             1844702
district_544M                        0
downpmt_134A                    123329
dtlastpmt_581D                 2860375
dtlastpmtallstes_3545839D      2434155
education_1138M                      0
employedfrom_700D              2180869
familystate_726L               1245201
firstnonzeroinstldate_307

In [7]:
data1.columns

Index(['case_id', 'actualdpd_943P', 'annuity_853A', 'approvaldate_319D',
       'byoccupationinc_3656910L', 'cancelreason_3545846M', 'childnum_21L',
       'creationdate_885D', 'credacc_actualbalance_314A',
       'credacc_credlmt_575A', 'credacc_maxhisbal_375A',
       'credacc_minhisbal_90A', 'credacc_status_367L',
       'credacc_transactions_402L', 'credamount_590A', 'credtype_587L',
       'currdebt_94A', 'dateactivated_425D', 'district_544M', 'downpmt_134A',
       'dtlastpmt_581D', 'dtlastpmtallstes_3545839D', 'education_1138M',
       'employedfrom_700D', 'familystate_726L', 'firstnonzeroinstldate_307D',
       'inittransactioncode_279L', 'isbidproduct_390L', 'isdebitcard_527L',
       'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'num_group1',
       'outstandingdebt_522A', 'pmtnum_8L', 'postype_4733339M',
       'profession_152M', 'rejectreason_755M', 'rejectreasonclient_4145042M',
       'revolvingaccount_394A', 'status_219L', 'tenor_203L'],
      dtype='object')

In [8]:
# dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

train_basetable = pd.read_csv(dataPath + "csv_files/train/train_base.csv")
train_static = pd.concat(
    [
        pd.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(convert_strings),
        pd.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(convert_strings),
    ],
    axis=0,
    ignore_index=True
)
train_static_cb = pd.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(convert_strings)
train_person_1 = pd.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(convert_strings)
train_credit_bureau_b_2 = pd.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(convert_strings)
test_basetable = pd.read_csv(dataPath + "csv_files/test/test_base.csv")
test_static = pd.concat(
    [
        pd.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(convert_strings),
        pd.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(convert_strings),
        pd.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(convert_strings),
    ],
    axis=0,
    ignore_index=True
)
test_static_cb = pd.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(convert_strings)
test_person_1 = pd.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(convert_strings)
test_credit_bureau_b_2 = pd.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv").pipe(convert_strings)

C:\Users\Peter\AppData\Local\Temp\ipykernel_9988\3419565707.py:16: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(convert_strings),
C:\Users\Peter\AppData\Local\Temp\ipykernel_9988\3419565707.py:17: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(convert_strings),
C:\Users\Peter\AppData\Local\Temp\ipykernel_9988\3419565707.py:22: DtypeWarning: Columns (1,2,3,4,7,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_cb = pd.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(convert_strings)
C:\Users\Peter\AppData\Local\Temp\ipykernel_9988\3419565707.py:23: DtypeWarning: Columns (16) have mixed types. Specify dtype option on impor

In [17]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.groupby("case_id").agg(
    {"mainoccupationinc_384A": "max",
     "incometype_1044T": lambda x: (x == "SELFEMPLOYED").max()}
).rename(columns={"mainoccupationinc_384A": "mainoccupationinc_384A_max",
                  "incometype_1044T": "mainoccupationinc_384A_any_selfemployed"})

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1[train_person_1["num_group1"] == 0][["case_id", "housetype_905L"]] \
    .rename(columns={"housetype_905L": "person_housetype"}).drop(columns=["num_group1"])

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.groupby("case_id").agg(
    {"pmts_pmtsoverdue_635A": "max",
     "pmts_dpdvalue_108P": lambda x: (x > 31).max()}
).rename(columns={"pmts_pmtsoverdue_635A": "pmts_pmtsoverdue_635A_max",
                  "pmts_dpdvalue_108P": "pmts_dpdvalue_108P_over31"})

# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = [col for col in train_static.columns if col[-1] in ("A", "M")]
print(selected_static_cols)

selected_static_cb_cols = [col for col in train_static_cb.columns if col[-1] in ("A", "M")]
print(selected_static_cb_cols)

# Join all tables together.
data = train_basetable.merge(train_static[["case_id"] + selected_static_cols], how="left", on="case_id") \
    .merge(train_static_cb[["case_id"] + selected_static_cb_cols], how="left", on="case_id") \
    .merge(train_person_1_feats_1, how="left", on="case_id") \
    .merge(train_person_1_feats_2, how="left", on="case_id") \
    .merge(train_credit_bureau_b_2_feats, how="left", on="case_id")


KeyError: "['num_group1'] not found in axis"

In [18]:
train_person_1

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,NaN,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,NaN,NaN,NaN,CL,NaN,True,F,PRIMARY_MOBILE
1,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,SPOUSE,NaN,False,EM,NaN,NaN,NaN,PHONE
2,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,NaN,NaN,NaN,PHONE
3,0,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,COLLEAGUE,NaN,PE,NaN,NaN,NaN,PHONE
4,1,1957-08-01,NaN,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,NaN,NaN,NaN,CL,NaN,True,M,PRIMARY_MOBILE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973986,2703451,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,NaN,NaN,CL,NaN,NaN,NaN,HOME_PHONE
2973987,2703452,1977-08-01,NaN,NaN,P133_44_167,False,False,P59_150_74,a55475b1,NaN,...,P19_11_176,P11_15_81,NaN,NaN,NaN,CL,NaN,False,M,PRIMARY_MOBILE
2973988,2703453,1950-02-01,NaN,NaN,P123_6_84,False,False,P46_103_143,a55475b1,NaN,...,P123_6_84,P46_103_143,NaN,NaN,NaN,CL,NaN,False,F,PRIMARY_MOBILE
2973989,2703453,NaN,NaN,NaN,a55475b1,NaN,NaN,a55475b1,a55475b1,NaN,...,a55475b1,a55475b1,NaN,NaN,NaN,CL,NaN,NaN,NaN,HOME_PHONE


In [9]:
train_person_1_feats_1

NameError: name 'train_person_1_feats_1' is not defined